# Interactive Componentes

## Single Callbacks for Interactivity

- Steps to create a callback from interactions:
    - Create a function to return some desired output;
    - Decorate that function with an @app.callback decorator;
        - Set an **Output** to a component id;
        - Set an **Input** to a component id.
    - Connect the desired properties.

In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [6]:
app = dash.Dash()

app.layout = html.Div(
    [
        dcc.Input(
            id = 'my-id',
            value = 'Initial Text',
            type = 'text',
        ),
        html.Div(
            id = 'my-div',
            style = {
                'border': '2px blue solid'
            }
        )
    ]
)

@app.callback(
    Output(
        component_id = 'my-div', 
        component_property = 'children'
    ),
    [
        Input(
            component_id = 'my-id',
            component_property = 'value'
        )
    ]
)
def update_output_div(input_value):
    return "Your entered: {}".format(input_value)

if __name__ == '__main__':
    app.run_server()

 * Serving Flask app "dash" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Jun/2022 16:39:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2022 16:39:04] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2022 16:39:04] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2022 16:39:04] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2022 16:39:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2022 16:39:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2022 16:39:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2022 16:39:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2022 16:39:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2022 16:39:10] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2022 16:39:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2022 16:39:11] "POST /_dash-update-component

## Dash Callbacks for Graphs

In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd

In [2]:
df = pd.read_csv('../Data/gapminderDataFiveYear.csv')

In [3]:
app = dash.Dash()


year_options = []
for year in df['year'].unique():
    year_options.append({'label': str(year), 'value':year})


app.layout = html.Div(
    [
        dcc.Graph(id = 'graph'),
        dcc.Dropdown(
            id = 'year-picker',
            options = year_options,
            value = df['year'].min()
        )
    ]
)

@app.callback(
    Output('graph', 'figure'),
    [Input(
        component_id = 'year-picker', 
        component_property = 'value'
    )]
)
def update_figure(selected_year):
    filtered_df = df[df['year'] == selected_year]

    traces = []

    for continent_name in filtered_df['continent'].unique():
        df_by_continent = filtered_df[filtered_df['continent'] == continent_name]
        traces.append(
            go.Scatter(
                x = df_by_continent['gdpPercap'],
                y = df_by_continent['lifeExp'],
                mode = 'markers',
                opacity = 0.7,
                marker = {'size': 15},
                name = continent_name
            )
        )
    return {'data': traces, 'layout': go.Layout(
        title = 'My Plot',
        xaxis = {'title': 'GDP Per Cap', 'type': 'log'},
        yaxis = {'title': 'Life Expectancy'}
    )}

if __name__ == '__main__':
    app.run_server()

 * Serving Flask app "dash" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Jun/2022 15:36:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Jun/2022 15:36:38] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [05/Jun/2022 15:36:38] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [05/Jun/2022 15:36:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Jun/2022 15:36:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Jun/2022 15:37:06] "POST /_dash-update-component HTTP/1.1" 200 -


In [4]:
year_options

[{'label': '1952', 'value': 1952},
 {'label': '1957', 'value': 1957},
 {'label': '1962', 'value': 1962},
 {'label': '1967', 'value': 1967},
 {'label': '1972', 'value': 1972},
 {'label': '1977', 'value': 1977},
 {'label': '1982', 'value': 1982},
 {'label': '1987', 'value': 1987},
 {'label': '1992', 'value': 1992},
 {'label': '1997', 'value': 1997},
 {'label': '2002', 'value': 2002},
 {'label': '2007', 'value': 2007}]

## Multiple Components

### Multiple inputs

In [25]:
df = pd.read_csv('../Data/mpg.csv')

In [16]:
app = dash.Dash()

features = df.columns

app.layout = html.Div(
    [
        html.Div(
            [
                dcc.Dropdown(
                    id = 'xaxis',
                    options = [
                        {'label': i, 'value': i} for i in features
                    ],
                    value = 'displacement'
                ),
            ],
            style = {'width': '48%', 'display': 'inline-block'}
        ),
        html.Div(
            [
                dcc.Dropdown(
                    id = 'yaxis',
                    options = [
                        {'label': i, 'value': i} for i in features
                    ],
                    value = 'mpg'
                )
            ],
            style = {'width': '48%', 'display': 'inline-block'}
        ),
        dcc.Graph(id = 'feature-graphic')
    ],
    style = {'padding': 10}
)

@app.callback(
    Output('feature-graphic', 'figure'),
    [Input('xaxis', 'value'), 
    Input('yaxis', 'value')]
)
def update_graph(xaxis_name, yaxis_name):
    return {
        'data': [
            go.Scatter(
                x = df[xaxis_name],
                y = df[yaxis_name],
                text = df['name'],
                mode = 'markers',
                marker = {
                    'size': 15,
                    'opacity': 0.5,
                    'line': {
                        'width': 0.5,
                        'color': 'white'
                    }
                }
            )
        ], 
        'layout': go.Layout(
            title = 'My Dashboard for MPG',
            xaxis = {'title': xaxis_name},
            yaxis = {'title': yaxis_name},
            hovermode = 'closest'
        )
    }

if __name__ == '__main__':
    app.run_server()

 * Serving Flask app "dash" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Jun/2022 16:05:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Jun/2022 16:05:27] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [05/Jun/2022 16:05:27] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [05/Jun/2022 16:05:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Jun/2022 16:05:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Jun/2022 16:05:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Jun/2022 16:05:35] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Jun/2022 16:05:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Jun/2022 16:05:38] "POST /_dash-update-component HTTP/1.1" 200 -


### Multiple Outputs

In [9]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
import base64

In [26]:
df = pd.read_csv('../Data/wheels.csv')

In [27]:
df.head(3)

,wheels,color,image
0,1,red,redunicycle.jpg
1,1,blue,blueunicycle.jpg
2,1,yellow,yellowunicycle.jpg


In [28]:
app = dash.Dash()

def encode_image(image_file):
    encoded = base64.b64encode(open(image_file, 'rb').read())
    return 'data:image/png;base64,{}'.format(encoded.decode())

app.layout = html.Div(
    [
        dcc.RadioItems(
            id = 'wheels',
            options = [
                {'label': i, 'value': i} for i in df.wheels.unique()
            ],
            value = 1
        ),
        html.Div(
            id = 'wheels-output'
        ),
        html.Hr(),
        dcc.RadioItems(
            id = 'colors',
            options = [
                {'label': i, 'value': i} for i in df.color.unique()
            ],
            value = 1
        ),        
        html.Div(
            id = 'colors-output'
        ),
        html.Img(
            id = 'display-image',
            src = 'children',
            height = 300,
        )
    ],
    style = {
        'fontFamily': 'helvetica', 
        'fontsize': 18
    }
)

@app.callback(
    Output(
        'wheels-output', 'children'
    ),
    [
        Input(
            'wheels',
            'value'
        )
    ]
)
def callback_wheels(wheels_value):
    return "you chose {}".format(wheels_value)

@app.callback(
    Output(
        'colors-output', 'children'
    ),
    [
        Input(
            'colors',
            'value'
        )
    ]
)
def callback_colors(colors_value):
    return "you chose {}".format(colors_value)


@app.callback(
    Output('display-image', 'src'),
    [
        Input(
            'wheels', 'value'
        ),
        Input(
            'colors', 'value'
        )
    ]
)
def callback_image(wheel, color):
    path = '../Data/Images/'
    return encode_image(path+df[(df['wheels']==wheel) & (df['color']==color)]['image'].values[0])

if __name__ == '__main__':
    app.run_server()

 * Serving Flask app "dash" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Jun/2022 19:29:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Jun/2022 19:29:32] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [05/Jun/2022 19:29:32] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [05/Jun/2022 19:29:32] "POST /_dash-update-component HTTP/1.1" 200 -
[2022-06-05 19:29:32,336] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/fabio/anaconda3/envs/dash/lib/python3.6/site-packages/flask/app.py", line 2317, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/fabio/anaconda3/envs/dash/lib/python3.6/site-packages/flask/app.py", line 1840, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/fabio/anaconda3/envs/dash/lib/python3.6/site-packages/flask/app.py", line 1743, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/fabio/anaconda3/envs/dash/lib/python3.6/site-packa